# Setup and Imports

In [1]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

In [2]:
#functions 
def get_movie_rating(movie_id):
    #movie and release date
    movie = tmdb.Movies(movie_id)
    #create output dict
    movie_info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1'] =='US':
            movie_info['certification'] = c['certification']
    return movie_info

def write_json(new_data, filename): 
    """Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""    
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [12]:
#confirm files from part 1 of project are in Data folder
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json']

In [11]:
#load basics csv into a dataframe
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head(3)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama


In [5]:
with open('/Users/wombatmaster/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
    
tmdb.API_KEY =  login['api-key']

In [6]:
YEARS_TO_GET = [2000,2001]

For loop to grab the rating for movies from 2000 and 2001


In [10]:
#start outer loop
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',position=0):
    #select JSON file to save progress
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    #create is needed
    file_exists = os.path.isfile(JSON_FILE)
    if file_exists == False:
            with open(JSON_FILE,'w') as f:
                json.dump([{'imdb_id':0}],f)
    #save new year             
    df = basics.loc[ basics['startYear']==YEAR].copy()
    #save movie ids
    movie_ids = df['tconst'].copy()
    #recall previous results
    previous_df = pd.read_json(JSON_FILE)
    #filter out movies that currently exist in the file
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        # Attempt to retrieve then data for the movie id
        try:
            temp = get_movie_with_rating(movie_id)  #This uses your pre-made function
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        # If it fails,  make a dict with just the id and None for certification.
        except Exception as e:
            continue
            
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)


YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1409 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1525 [00:00<?, ?it/s]

In [8]:
results_2000 = pd.read_csv("Data/final_tmdb_data_2000.csv.gz")
results_2000.head()

,imdb_id
0,0


In [9]:
results_2001 = pd.read_csv("Data/final_tmdb_data_2000.csv.gz")
results_2001.head()

,imdb_id
0,0
